In [32]:
import numpy as np
import pandas as pd


In [33]:
file_path = 'hist.xlsx'

In [34]:
df = pd.read_excel(file_path)

In [36]:
print(df.head(10))

   week_id  customer_id attribute1  state_id Sex  campaign_id  response
0        1            1          B         2   M            1         1
1        1            2          A        38   F            2         0
2        1            3          C        46   M            3         0
3        1            4          B        35   M            4         0
4        1            5          B        22   M            5         1
5        1            6          B        39   F            6         1
6        1            7          A        28   M            7         1
7        1            8          A        46   M            8         0
8        1            9          C        32   M            9         1
9        1           10          C        25   M           10         1


In [49]:
print(df['campaign_id'])

0         1
1         2
2         3
3         4
4         5
         ..
259995    1
259996    2
259997    3
259998    4
259999    5
Name: campaign_id, Length: 260000, dtype: int64


What do we know? 
Smartmarket has 10 campaigns to send to it subscribers, and it continues until all campaigns are delivered to the sanme number of subscribers.

Week_id is when campaign was delivered
customer_id is the subscriber
attribute1 is different users category
state_id is where subscriber is located
Sex is the gender of the subscriber
campaign_id is the campaign delivered to the subsriber
response is if the subscriber responded to the campaign

### Let's calculate the response rate for each subscriber from weeks 1 to 26

In [54]:
response_summary = df.groupby('customer_id')['response'].sum().reset_index()
response_summary.columns = ['customer_id', 'total_responses']
print(response_summary.head(11))

    customer_id  total_responses
0             1                6
1             2               10
2             3               14
3             4                1
4             5                4
5             6                7
6             7               11
7             8               23
8             9               18
9            10               16
10           11               23


### Next we are going to rank the subscribers based on response rates and sort the subscribers by total responses and select the top 25%

In [55]:
top_25_percent_count = int(len(response_summary) * 0.25)
top_subscribers = response_summary.sort_values(by='total_responses', ascending=False).head(top_25_percent_count)
print(top_subscribers.head(25))

      customer_id  total_responses
387           388               26
553           554               26
828           829               26
45             46               26
6453         6454               26
9893        13094               26
4338         4339               26
1007         1008               25
4037         4038               25
6988         6989               25
8488         8489               25
3989         3990               25
2377         2378               25
1918         1919               25
9185         9186               25
5731         5732               25
8729         8730               25
8943         8944               25
5139         5140               25
2142         2143               25
8347         8348               25
5116         5117               25
1739         1740               25
2586         2587               25
7555         7556               25


### Next we are going to extract the latest campaign from week 26 and going to rotate the campaigns based on the last campaign they received

In [68]:
week_26_data = df[df['week_id'] == 26][['customer_id', 'campaign_id']]
print(week_26_data)

        customer_id  campaign_id
250000            1            6
250001            2            7
250002            3            8
250003            4            9
250004            5           10
...             ...          ...
259995        13196            1
259996        13197            2
259997        13198            3
259998        13199            4
259999        13200            5

[10000 rows x 2 columns]


### Now we are going to merge the top subscribers with their week 26 campaign

In [67]:
top_subscribers_week_26 = top_subscribers.merge(week_26_data, on='customer_id')
print(top_subscribers_week_26)

      customer_id  total_responses  campaign_id
0             388               26            3
1             554               26            9
2             829               26            4
3              46               26            1
4            6454               26            9
...           ...              ...          ...
2495         2108               17            3
2496         3873               17            8
2497         6546               17            1
2498         8297               17            2
2499        12882               17            7

[2500 rows x 3 columns]


### Now we are going to assign campaigns for week 27 based on the rotation pattern

In [70]:
### Increment campaign number by 1, and if it's 10, rotate to 1
top_subscribers_week_26['week_27_campaign'] = top_subscribers_week_26['campaign_id'] % 10 + 1
print(top_subscribers_week_26)

      customer_id  total_responses  campaign_id  week_27_campaign
0             388               26            3                 4
1             554               26            9                10
2             829               26            4                 5
3              46               26            1                 2
4            6454               26            9                10
...           ...              ...          ...               ...
2495         2108               17            3                 4
2496         3873               17            8                 9
2497         6546               17            1                 2
2498         8297               17            2                 3
2499        12882               17            7                 8

[2500 rows x 4 columns]


### Show the results with assigned campaigns for week 27

In [74]:
top_subscribers_week_26[['customer_id', 'campaign_id', 'week_27_campaign']].head(25)

,customer_id,campaign_id,week_27_campaign
0,388,3,4
1,554,9,10
2,829,4,5
3,46,1,2
4,6454,9,10
5,13094,9,10
6,4339,4,5
7,1008,3,4
8,4038,3,4
9,6989,4,5


## Response Rate:
### To calculate the response rate for the top 25% subsribers, we are going to merge the progonal data with the top subscibers

In [77]:
top_subscribers_data = df[df['customer_id'].isin(top_subscribers['customer_id'])]

### Next we are going to calculate the total number of responses and total campaigns delivered for top subscribers

In [78]:
total_responses_top = top_subscribers_data['response'].sum()
total_campaigns_top = len(top_subscribers_data)

### And finally calculate the distorical response rate to show the response rate for the top 25%

In [79]:
historical_response_rate_top = total_responses_top / total_campaigns_top
historical_response_rate_top

0.7538153846153847